[View in Colaboratory](https://colab.research.google.com/github/zacateras/deep-siamese-text-similarity/blob/wedt/siamese_semantic.ipynb)

In [1]:
# If python3 is not loaded kill the hypervisor 
# ! kill -9 -1
import sys
sys.version

'2.7.14 (default, Sep 23 2017, 22:06:14) \n[GCC 7.2.0]'

In [2]:
! git clone -b wedt https://github.com/zacateras/deep-siamese-text-similarity.git

Cloning into 'deep-siamese-text-similarity'...
remote: Counting objects: 141, done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 141 (delta 12), reused 25 (delta 6), pack-reused 107
Receiving objects: 100% (141/141), 2.69 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [3]:
% cd ./deep-siamese-text-similarity
! pip install -r requirements.txt > pip.log

/content/deep-siamese-text-similarity
opencv-python 3.4.0.12 has requirement numpy>=1.11.1, but you'll have numpy 1.11.0 which is incompatible.
tensorboard 1.7.0 has requirement numpy>=1.12.0, but you'll have numpy 1.11.0 which is incompatible.


In [16]:
! rm -r data
! tools/download.w2v.en.simple.sh
! tools/download.sts.sh

--2018-05-19 22:25:26--  https://www.dropbox.com/s/sg4yxvzg5rysruy/wiki.simple.vec.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/oA6aL9DtpuwufEfl7S27s4TWfz7xuiZplPiuxm5uaxMHw11HWOHarxPtsyu4hS6C/file [following]
--2018-05-19 22:25:28--  https://dl.dropboxusercontent.com/content_link/oA6aL9DtpuwufEfl7S27s4TWfz7xuiZplPiuxm5uaxMHw11HWOHarxPtsyu4hS6C/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.9.6, 2620:100:601f:6::a27d:906
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.9.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108805476 (104M) [application/zip]
Saving to: ‘/content/deep-siamese-text-similarity/data/wiki.simple.vec.zip’

/content/deep-siame 100%[=============

In [ ]:
! tools/train_all.sh

In [4]:
import os

In [42]:
with open('runs/evaluation_combined.tsv', 'w') as evaluation_combined_file:
    for run in os.listdir('./runs'):
        if os.path.isdir('runs/%s' % run):
            print('Processing %s...' % run)
            with open('runs/%s/evaluation.tsv' % run, 'r') as evaluation_file:
                for line in evaluation_file:
                    evaluation_combined_file.write('%s\t%s' % (run, line))

Processing snli_1n-500,250,100_1d-1.0_2n-500,250,100_2d-1.0...
Processing snli_tied_1n-150,100,50_1d-1.0_2n-150,100,50_2d-1.0...
Processing snli_tied_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0...
Processing sick2014_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0...
Processing snli_tied_1n-150,100,50_1d-0.8_2n-150,100,50_2d-0.8...
Processing snli_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0...
Processing snli_tied_1n-500,250,100_1d-1.0_2n-500,250,100_2d-1.0...
Processing snli_1n-150,100,50_1d-0.8_2n-150,100,50_2d-0.8...
Processing snli_1n-150,100,50_1d-1.0_2n-150,100,50_2d-1.0...


In [62]:
import pandas as pd

df = pd.read_csv('runs/evaluation_combined.tsv', sep='\t', header = None, names = ['model', 'dataset', 'pcc', 'rho', 'mse5'])
df['mse1'] = df['mse5'] / 5.0
df = df.drop(columns = ['rho'])

In [63]:
df.sort_values(by = ['mse1'], ascending = True).head(10)

,model,dataset,pcc,mse5,mse1
22,"snli_tied_1n-150,100,50_1d-1.0_2n-150,100,50_2...",semeval2015-sts-images,0.667194,1.111719,0.222344
19,"snli_tied_1n-150,100,50_1d-1.0_2n-150,100,50_2...",semeval2015-sts-answers-students,0.423141,1.154750,0.230950
31,"snli_tied_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0",semeval2015-sts-answers-students,0.427069,1.158849,0.231770
34,"snli_tied_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0",semeval2015-sts-images,0.665915,1.171034,0.234207
71,"snli_tied_1n-500,250,100_1d-1.0_2n-500,250,100...",semeval2017-sts-track5,0.468057,1.191889,0.238378
81,"snli_tied_1n-500,250,100_1d-1.0_2n-500,250,100...",semeval2015-sts-images,0.671226,1.192194,0.238439
35,"snli_tied_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0",sick2014,0.468598,1.205824,0.241165
95,"snli_1n-150,100,50_1d-1.0_2n-150,100,50_2d-1.0",semeval2017-sts-track5,0.274367,1.209585,0.241917
24,"snli_tied_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0",semeval2017-sts-track5,0.472038,1.212033,0.242407
12,"snli_tied_1n-150,100,50_1d-1.0_2n-150,100,50_2...",semeval2017-sts-track5,0.466301,1.218704,0.243741


In [64]:
df.describe()

,pcc,mse5,mse1
count,107.000000,107.000000,107.000000
mean,0.207462,1.518006,0.303601
std,0.172192,0.196408,0.039282
min,-0.144816,1.111719,0.222344
25%,0.071560,1.397772,0.279554
50%,0.186899,1.513038,0.302608
75%,0.317532,1.672787,0.334557
max,0.671226,1.983374,0.396675


In [67]:
df.groupby('model').mean().sort_values(by = ['mse1'], ascending = True)

,pcc,mse5,mse1
model,,,
"snli_tied_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0",0.352802,1.420061,0.284012
"snli_tied_1n-150,100,50_1d-1.0_2n-150,100,50_2d-1.0",0.352494,1.431450,0.286290
"snli_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0",0.142370,1.433100,0.286620
"snli_1n-150,100,50_1d-1.0_2n-150,100,50_2d-1.0",0.110528,1.494837,0.298967
"snli_1n-150,100,50_1d-0.8_2n-150,100,50_2d-0.8",0.103233,1.544317,0.308863
"snli_tied_1n-500,250,100_1d-1.0_2n-500,250,100_2d-1.0",0.312196,1.547986,0.309597
"snli_tied_1n-150,100,50_1d-0.8_2n-150,100,50_2d-0.8",0.284520,1.563654,0.312731
"snli_1n-500,250,100_1d-1.0_2n-500,250,100_2d-1.0",-0.005859,1.584315,0.316863
"sick2014_1n-50,50,50_1d-1.0_2n-50,50,50_2d-1.0",0.215552,1.653634,0.330727


In [66]:
df.groupby('dataset').mean().sort_values(by = ['mse1'], ascending = True)

,pcc,mse5,mse1
dataset,,,
semeval2017-sts-track5,0.314622,1.314025,0.262805
semeval2015-sts-answers-students,0.254286,1.314701,0.262940
semeval2015-sts-images,0.411807,1.338057,0.267611
sick2014,0.315402,1.420584,0.284117
semeval2015-sts-belief,0.124022,1.502295,0.300459
semeval2015-sts-headlines,0.174499,1.526678,0.305336
semeval2015-sts-answers-forums,0.024932,1.546103,0.309221
semeval2016-sts-headlines,0.188816,1.566602,0.313320
semeval2016-sts-question-question,0.143488,1.589176,0.317835
